<a href="https://colab.research.google.com/github/LaJeremi/Tensorflow-nlp-tutorial-Practice-/blob/main/13.%20Subword%20Tokenizer/%2013_02_SentencePiece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 13-02 센텐스피스(SentencePiece)

BPE를 포함하여 기타 서브워드 토크나이징 알고리즘들을 내장한 센텐스피스(SentencePiece)는 일반적으로 실무에서 선택할 수 있는 최선의 선택 중 하나

# 1. 센텐스피스(SentencePiece)


https://wikidocs.net/86657

# 2. IMDB 리뷰 토큰화하기


In [ ]:
pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.1 MB/s eta 0:00:00


In [ ]:
pip list | grep sentencepiece


sentencepiece                 0.1.97


In [ ]:

import sentencepiece as spm
import pandas as pd
import urllib.request
import csv
     

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")


('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f86e860e9a0>)

In [ ]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [ ]:
print('리뷰 개수 :',len(train_df)) # 리뷰 개수 출력


리뷰 개수 : 50000


In [ ]:

with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

## * 센텐스피스로 단어 집합과 각 단어에 고유한 정수를 부여해보겠습니다.

In [ ]:
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')


* input : 학습시킬 파일

* model_prefix : 만들어질 모델 이름

* vocab_size : 단어 집합의 크기

* model_type : 사용할 모델 (unigram(default), bpe, char, word)

* max_sentence_length: 문장의 최대 길이
* pad_id, pad_piece: pad token id, 값
* unk_id, unk_piece: unknown token id, 값
* bos_id, bos_piece: begin of sentence token id, 값
* eos_id, eos_piece: end of sequence token id, 값
* user_defined_symbols: 사용자 정의 토큰

In [ ]:

vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
372,▁most,-369
3589,▁Hitchcock,-3586
348,ue,-345
2847,▁pace,-2844
2808,▁ess,-2805
535,ought,-532
4277,▁70',-4274
1150,▁sing,-1147
4230,▁inspired,-4227
1809,ement,-1806


In [ ]:
len(vocab_list)


5000

위 에서 vocab_size의 인자를 통해 단어 집합의 크기를 5,000개로 제한하였으므로 단어 집합의 크기는 5,000개



In [ ]:
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)
     

True

# encode_as_pieces : 문장을 입력하면 서브 워드 시퀀스로 변환합니다.

# encode_as_ids : 문장을 입력하면 정수 시퀀스로 변환합니다.

In [ ]:

lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()
     

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]



# GetPieceSize() : 단어 집합의 크기를 확인합니다.


In [ ]:
sp.GetPieceSize()


5000

# idToPiece : 정수로부터 맵핑되는 서브 워드로 변환합니다.


In [ ]:
sp.IdToPiece(430)


'▁character'

# PieceToId : 서브워드로부터 맵핑되는 정수로 변환합니다.


In [ ]:
sp.PieceToId('▁character')


430

# DecodeIds : 정수 시퀀스로부터 문장으로 변환합니다.


In [ ]:
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91])


'I have waited a long time for someone to film'

# DecodePieces : 서브워드 시퀀스로부터 문장으로 변환합니다.


In [ ]:
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film'])


'I have waited a long time for someone to film'

# encode : 문장으로부터 인자값에 따라서 정수 시퀀스 또는 서브워드 시퀀스로 변환 가능합니다.


In [ ]:

print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]


# 3. 네이버 영화 리뷰 토큰화하기


In [ ]:

import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")


('ratings.txt', <http.client.HTTPMessage at 0x7f86b9f3ad90>)

In [ ]:
naver_df = pd.read_table('ratings.txt')
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [ ]:
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력


리뷰 개수 : 200000


In [ ]:
print(naver_df.isnull().values.any())


True


In [ ]:

naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [ ]:
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력


리뷰 개수 : 199992


In [ ]:

with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [ ]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')


In [ ]:

vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,▁그,-6


In [ ]:
vocab_list.sample(10)


,0,1
619,▁대사,-616
4979,눔,-4976
2603,▁나왔으면,-2600
3027,▁언제나,-3024
2636,이의,-2633
739,▁웃기,-736
1918,▁좋았어요,-1915
924,▁배우들의,-921
3194,▁고통,-3191
3979,츠,-3976


In [ ]:
len(vocab_list)


5000

In [ ]:

sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

True

In [ ]:

lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()
     

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[132, 966, 1296, 2590, 3276]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]



# GetPieceSize() : 단어 집합의 크기를 확인합니다.


In [ ]:
sp.GetPieceSize()


5000

# idToPiece : 정수로부터 맵핑되는 서브 워드로 변환합니다.


In [ ]:
sp.IdToPiece(4)


'영화'

# PieceToId : 서브워드로부터 맵핑되는 정수로 변환합니다.


In [ ]:
sp.PieceToId('영화')


4

# DecodeIds : 정수 시퀀스로부터 문장으로 변환합니다.


In [ ]:
sp.DecodeIds([54, 200, 821, 85])


'진짜 최고의 영화입니다 ᄏᄏ'

# DecodePieces : 서브워드 시퀀스로부터 문장으로 변환합니다.


In [ ]:
sp.DecodePieces(['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ'])


'진짜 최고의 영화입니다 ᄏᄏ'

# encode : 문장으로부터 인자값에 따라서 정수 시퀀스 또는 서브워드 시퀀스로 변환 가능합니다.


In [ ]:

print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]
